### async

In [1]:
import asyncio

async def f1():
    return 10

async def main():
    # f1 it's a coroutine object, and will not really run f1() function
    print(f1()) # <coroutine object f1 at 0x0000000004EC1548>
    
    # wait until f1() is returned
    print(await f1()) # 10
    
    # f1() will be scheduled and run immediately
    task1 = asyncio.create_task(f1())
    # wait until f1() is returned
    print(await task1) # 10

loop = asyncio.get_event_loop()
loop.create_task(main())

<Task pending coro=<main() running at <ipython-input-1-8f626a37c2e5>:6>>

<coroutine object f1 at 0x0000000004EDB4C8>
10
10


C:\Users\Hugo\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: coroutine 'f1' was never awaited
  


### create_task()

In [2]:
import asyncio


async def f1():
    print("f1")
    await asyncio.sleep(1)
    print('f1 done')
    
async def f2():
    print("f2")
    await asyncio.sleep(1)
    print('f2 done')

loop = asyncio.get_event_loop()
loop.create_task(f1())
loop.create_task(f2())
# when create_task() is called, f1 & f2 function will be scheduled and run immediately

<Task pending coro=<f2() running at <ipython-input-2-517908005102>:9>>

f1
f2
f1 done
f2 done


### await

In [3]:
import asyncio

async def f1():
    print("f1")
    await asyncio.sleep(1) # if you want to f1() & f2() run simultaneously, you need to use asyncio.sleep()
    print('f1 done')
    
async def f2():
    print("f2")
    await asyncio.sleep(1) # if you want to f1() & f2() run simultaneously, you need to use asyncio.sleep()
    print('f2 done')

async def main():
    task1 = asyncio.create_task(f1())
    task2 = asyncio.create_task(f2())
    
    # await must be used inside the async function
    # await will wait task1 completed
    await task1
    await task2

loop = asyncio.get_event_loop()
loop.create_task(main())

<Task pending coro=<main() running at <ipython-input-3-e950c2ea41b5>:13>>

f1
f2
f1 done
f2 done


### gather

In [4]:
import asyncio

async def f1(x):
    return x * 2

async def main():
    ret = await asyncio.gather(f1(2), f1(3), f1(4))
    print(ret) # [4, 6, 8]

loop = asyncio.get_event_loop()
loop.create_task(main())

<Task pending coro=<main() running at <ipython-input-4-8961fe24879a>:6>>

[4, 6, 8]


### wait_for

In [5]:
import asyncio

async def f1():
    print("f1()")
    await asyncio.sleep(2)
    print("f1() after") # will not come here
    
async def main():
    try:
        await asyncio.wait_for(f1(), timeout=1.0)
        print("wait_for() after") # will not come here
    except asyncio.TimeoutError:
        print("timeout")

loop = asyncio.get_event_loop()
loop.create_task(main())

<Task pending coro=<main() running at <ipython-input-5-51339c87147f>:8>>

f1()
timeout


### TCP Server

In [6]:
import asyncio, traceback

async def echo_handler(reader, writer):
    addr = writer.get_extra_info('peername')
    print(addr, 'connected')
    while True:
        data = await reader.read(100)
        if len(data) == 0:
            print(addr, 'client disconnected')
            writer.close()
            await writer.wait_closed()
            break
        else:
            print(addr, 'read data=', data)
            writer.write(data)
            await writer.drain()
    
async def tcp_echo_server():
    try:
        server = await asyncio.start_server(echo_handler, '127.0.0.1', 12345)
        await server.serve_forever() # block here
    except Exception as err:
        print(traceback.format_exc())

loop = asyncio.get_event_loop()
loop.create_task(tcp_echo_server())

<Task pending coro=<tcp_echo_server() running at <ipython-input-6-89195b827ebc>:18>>

### TCP Client

In [7]:
import asyncio, traceback

async def tcp_echo_client(message):
    try:
        reader, writer = await asyncio.open_connection('127.0.0.1', 12345)
        await asyncio.sleep(1)
        
        print('write')
        writer.write(message.encode())
        
        await asyncio.sleep(1)
        data = await reader.read(100)
        print('read=', data)

        await asyncio.sleep(1)
        print('close')
        writer.close()
        await writer.wait_closed()
    except Exception as err:
        print(traceback.format_exc())

loop = asyncio.get_event_loop()
loop.create_task(tcp_echo_client("hugo test 1"))
loop.create_task(tcp_echo_client("hugo test 2"))

<Task pending coro=<tcp_echo_client() running at <ipython-input-7-05f970d62051>:3>>

('127.0.0.1', 53846) connected
('127.0.0.1', 53847) connected
write
write
('127.0.0.1', 53846) read data= b'hugo test 1'
('127.0.0.1', 53847) read data= b'hugo test 2'
read= b'hugo test 1'
read= b'hugo test 2'
close
close
('127.0.0.1', 53846) client disconnected
('127.0.0.1', 53847) client disconnected


### Queue

In [8]:
async def consumer():
    for i in range(3):
        ret = await queue.get() # will be blocked if queue is empty
        print('get=', ret)
    
async def producer():
    for i in range(3):
        print('put=', i)
        await queue.put(i) # will be blocked if queue is full

queue = asyncio.Queue(2)
print(queue)
    
loop = asyncio.get_event_loop()
loop.create_task(producer())
loop.create_task(consumer())

<Queue maxsize=2>


<Task pending coro=<consumer() running at <ipython-input-8-71ddff9a1e10>:1>>

put= 0
put= 1
put= 2
get= 0
get= 1
get= 2


### TCP Server + Client + Queue

In [9]:
import asyncio, traceback

# -------------- server -------------- 
async def echo_handler(reader, writer):
    addr = writer.get_extra_info('peername')
    print(addr, 'connected')
    while True:
        data = await reader.read(100)
        if len(data) == 0:
            print(addr, 'client disconnected')
            writer.close()
            await writer.wait_closed()
            
            await queue.put('%s disconnected' % (str(addr)))
            break
        else:
            print(addr, 'read data=', data)
            writer.write(data)
            await writer.drain()
    
async def tcp_echo_server():
    try:
        server = await asyncio.start_server(echo_handler, '127.0.0.1', 23456)
        await server.serve_forever() # block here
    except Exception as err:
        print(traceback.format_exc())

# -------------- client -------------- 
async def tcp_echo_client(message):
    try:
        reader, writer = await asyncio.open_connection('127.0.0.1', 23456)
        #await asyncio.sleep(0.5)
        
        print('client write')
        writer.write(message.encode())
        
        #await asyncio.sleep(0.5)
        data = await reader.read(100)
        print('client read=', data)

        #await asyncio.sleep(0.5)
        print('client close')
        writer.close()
        await writer.wait_closed()
        
        await queue.put('client close success')
    except Exception as err:
        print(traceback.format_exc())

# -------------- queue event -------------- 

async def queue_event():
    while True:
        ret = await queue.get()
        print('queue get=', ret)
        

# -------------- main -------------- 
queue = asyncio.Queue(10)

loop = asyncio.get_event_loop()
loop.create_task(tcp_echo_server())
loop.create_task(queue_event())
loop.create_task(tcp_echo_client("hugo test 1"))
loop.create_task(tcp_echo_client("hugo test 2"))

<Task pending coro=<tcp_echo_client() running at <ipython-input-9-95223f15aaee>:29>>

('127.0.0.1', 53869) connected
('127.0.0.1', 53870) connected
client write
client write
('127.0.0.1', 53869) read data= b'hugo test 1'
('127.0.0.1', 53870) read data= b'hugo test 2'
client read= b'hugo test 2'
client close
client read= b'hugo test 1'
client close
queue get= client close success
queue get= client close success
('127.0.0.1', 53869) client disconnected
('127.0.0.1', 53870) client disconnected
queue get= ('127.0.0.1', 53869) disconnected
queue get= ('127.0.0.1', 53870) disconnected
